# Hypothesis Testing Movie Info

In [1]:
# Import Libraries 
import pandas as pd
import numpy as np
import os, json, time
from tqdm.notebook import tqdm_notebook 

## Save json files as csv files / dataframes

In [6]:
YEARS_TO_GET = list(range(2010,2021))
api_df = pd.DataFrame()

In [8]:
for year in YEARS_TO_GET:
    # select JSON_FILE name and check if file exists
    JSON_FILE = f'Data/tmdb_api_results_{year}.json'
    
    # convert json file to .csv.gz
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'Data/final_tmdb_data_{year}.csv.gz', 
                         compression='gzip',
                        index=False)
    
    api_df = pd.concat([api_df, final_year_df], ignore_index=True)

In [9]:
api_df.info()
api_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40570 entries, 0 to 40569
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                40570 non-null  object 
 1   adult                  40559 non-null  float64
 2   backdrop_path          26186 non-null  object 
 3   belongs_to_collection  2384 non-null   object 
 4   budget                 40559 non-null  float64
 5   genres                 40559 non-null  object 
 6   homepage               37998 non-null  object 
 7   id                     40559 non-null  float64
 8   original_language      40559 non-null  object 
 9   original_title         40559 non-null  object 
 10  overview               40559 non-null  object 
 11  popularity             40559 non-null  float64
 12  poster_path            37225 non-null  object 
 13  production_companies   40559 non-null  object 
 14  production_countries   40559 non-null  object 
 15  re

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0312305,0.0,None,None,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,...,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Quantum Quest: A Cassini Space Odyssey,0.0,8.4,7.0,
2,tt0326965,0.0,/xt2klJdKCVGXcoBGQrGfAS0aGDE.jpg,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.inmysleep.com,40048.0,en,In My Sleep,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sleepwalking Can Be Deadly,In My Sleep,0.0,5.5,31.0,PG-13
3,tt0331312,0.0,None,None,0.0,[],,214026.0,en,This Wretched Life,...,0.0,0.0,[],Released,,This Wretched Life,0.0,5.0,1.0,
4,tt0393049,0.0,/gc9FN5zohhzCt05RkejQIIPLtBl.jpg,None,300000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,324352.0,en,Anderson's Cross,...,0.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes the boy next door is more than the b...,Anderson's Cross,0.0,4.0,5.0,
